# Web Scraping

Este notebook é utilizado para ler os dados da URL https://petsrs.com.br , lá se encontra um banco de dados de pets registrados. Ao clicar em cada item/imagem do pet no site é direcionado aos detalhes do pet (https://petsrs.com.br/pet/10), onde, 10 é o id do pet. Dentro dos detalhes tem o nome, status, abrigo, detalhes e URL da imagem do pet. Este notebook, performa um loop do id 1000 ao 2710 e extrai estas informações e salva localmente em um .xlsx.

Alguns detalhes mudam e as vezes vem valores inválidos, o xlsx resultante final deve ser limpo. 
A ideia é usar as URL dos pets e baixa-las. Unir com as demais informações, limpar o dataset e iniciar o treinamento dos modelos de IA.

In [5]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from openpyxl import Workbook
from openpyxl import load_workbook
import time

def extrair_detalhes_pet(url):
    # Configuração do Selenium para usar um driver do Chrome
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')  # Para executar o Chrome em modo headless (sem interface gráfica)
    driver = webdriver.Chrome(options=options)
    
    # Carregar a página
    driver.get(url)
    
    # Obter o código HTML da página após a renderização completa
    page_html = driver.page_source
    
    # Fechar o driver do Selenium
    driver.quit()
    
    # Parsear o HTML com BeautifulSoup
    soup = BeautifulSoup(page_html, 'html.parser')
    
    # Extrair detalhes do pet
    nome_pet = soup.find('div', class_='v-col-md-8').find('span').get_text(strip=True)
    
    status_pet_tag = soup.find('div', class_='v-chip__content')
    status_pet = status_pet_tag.get_text(strip=True) if status_pet_tag else 'Não encontrado'
    
    descricao_pet_tag = soup.find('div', class_='v-alert__content')
    descricao_pet = descricao_pet_tag.get_text(strip=True) if descricao_pet_tag else 'Não encontrado'
    
    abrigo_presente, nome_abrigo, endereco_abrigo = verificar_abrigo(page_html)
    
    url_imagem = extrair_url_imagem(url)
    
    return {'Nome do Pet': nome_pet, 'Status do Pet': status_pet, 'Descrição do Pet': descricao_pet, 
            'Abrigo': nome_abrigo if abrigo_presente else 'Não encontrado', 'Endereço do Abrigo': endereco_abrigo if abrigo_presente else 'Não encontrado', 
            'URL': url, 'URL da Imagem': url_imagem}

def extrair_url_imagem(url):
    # Configuração do Selenium para usar um driver do Chrome
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')  # Para executar o Chrome em modo headless (sem interface gráfica)
    driver = webdriver.Chrome(options=options)
    
    # Carregar a página
    driver.get(url)
    
    # Obter o código HTML da página após a renderização completa
    page_html = driver.page_source
    
    # Fechar o driver do Selenium
    driver.quit()
    
    # Parsear o HTML com BeautifulSoup
    soup = BeautifulSoup(page_html, 'html.parser')
    
    # Encontrar a tag img com a classe v-img__img e extrair a URL da imagem
    img_tag = soup.find('img', class_='v-img__img')
    if img_tag:
        return img_tag['src']
    else:
        return None
    
def extrair_url_imagem_timer(url):
    # Configuração do Selenium para usar um driver do Chrome
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')  # Para executar o Chrome em modo headless (sem interface gráfica)
    driver = webdriver.Chrome(options=options)
    
    # Carregar a página
    driver.get(url)
    
    # Esperar por um curto período para permitir o carregamento completo da página
    time.sleep(4)  # Aguarda 1 segundo (você pode ajustar esse valor conforme necessário)
    
    # Obter o código HTML da página após a renderização completa
    page_html = driver.page_source
    
    # Fechar o driver do Selenium
    driver.quit()
    
    # Parsear o HTML com BeautifulSoup
    soup = BeautifulSoup(page_html, 'html.parser')
    
    # Encontrar a tag img com a classe v-img__img e extrair a URL da imagem
    img_tag = soup.find('img', class_='v-img__img')
    if img_tag:
        return img_tag['src']
    else:
        return None

def verificar_abrigo(html):
    soup = BeautifulSoup(html, 'html.parser')
    
    abrigo_info = soup.find('div', class_='v-list-item-title', string=lambda string: string and string.startswith('Abrigo:'))
    if abrigo_info:
        abrigo = abrigo_info.text.strip().replace('Abrigo:', '').strip()
        endereco = abrigo_info.find_next('div', class_='v-list-item-subtitle').text.strip()
        return True, abrigo, endereco
    
    return False, None, None

# Cria ou carrega o arquivo Excel
try:
    wb = load_workbook('databases.xlsx')
except FileNotFoundError:
    wb = Workbook()
    ws = wb.active
    ws.append(['Nome do Pet', 'Status do Pet', 'Descrição do Pet', 'Abrigo', 'Endereço do Abrigo', 'URL', 'URL da Imagem'])
else:
    ws = wb.active

# Lista para armazenar os dados de cada pet
dados_pets = []

# Varrendo os pets 10, 2710 # do 1000 ao 2710 demorou 510min
for pet_id in range(1000, 2710):
    url_pet = f'https://petsrs.com.br/pet/{pet_id}'
    try:
        detalhes_pet = extrair_detalhes_pet(url_pet)
        dados_pets.append(detalhes_pet)
        ws.append([detalhes_pet['Nome do Pet'], detalhes_pet['Status do Pet'], detalhes_pet['Descrição do Pet'],
                   detalhes_pet['Abrigo'], detalhes_pet['Endereço do Abrigo'], detalhes_pet['URL'], detalhes_pet['URL da Imagem']])
        print(f"Pet ID {pet_id} salvo com sucesso.")
    except Exception as e:
        print(f"Erro ao salvar pet ID {pet_id}: {str(e)}")

# Salva as alterações no arquivo Excel
wb.save('databases.xlsx')


Pet ID 1000 salvo com sucesso.
Pet ID 1001 salvo com sucesso.
Pet ID 1002 salvo com sucesso.
Pet ID 1003 salvo com sucesso.
Pet ID 1004 salvo com sucesso.
Pet ID 1005 salvo com sucesso.
Pet ID 1006 salvo com sucesso.
Pet ID 1007 salvo com sucesso.
Pet ID 1008 salvo com sucesso.
Pet ID 1009 salvo com sucesso.
Pet ID 1010 salvo com sucesso.
Pet ID 1011 salvo com sucesso.
Pet ID 1012 salvo com sucesso.
Pet ID 1013 salvo com sucesso.
Pet ID 1014 salvo com sucesso.
Pet ID 1015 salvo com sucesso.
Pet ID 1016 salvo com sucesso.
Pet ID 1017 salvo com sucesso.
Pet ID 1018 salvo com sucesso.
Pet ID 1019 salvo com sucesso.
Pet ID 1020 salvo com sucesso.
Pet ID 1021 salvo com sucesso.
Pet ID 1022 salvo com sucesso.
Pet ID 1023 salvo com sucesso.
Pet ID 1024 salvo com sucesso.
Pet ID 1025 salvo com sucesso.
Pet ID 1026 salvo com sucesso.
Pet ID 1027 salvo com sucesso.
Pet ID 1028 salvo com sucesso.
Pet ID 1029 salvo com sucesso.
Pet ID 1030 salvo com sucesso.
Pet ID 1031 salvo com sucesso.
Pet ID 1